### WEB Scrapping with Selenium:
    we are going to scrap the cars data from the olx using the selenium library.
    What type of data we are going to collect from the olx??
    since the price of the used car strongly depends on the below independent variables:
        1.name of car company: the resale value of a used car gratly dependent on the company name.
        2.car model: of same company the prices are different for differenet cars
        3.car manufacturing year:with increasing the manufacturing year the value of car will decrease.
        4.location and area: same model car will have different resale value based on the state, area in which it is selling
        5.Kilometers Driven: more the kilometers driver less the engine life, spare parts etc,. so more kilometers driven less the cost
    the above are the some of attributes or futures we are going to collect along with the price of the car from olx.


In [40]:
#importing required libraries for web scraping with selenium
from selenium import webdriver
import pandas as pd
import selenium
import warnings
import time
warnings.filterwarnings('ignore')
import selenium.common.exceptions as exc

In [58]:
#Chrome web driver location provided for selenium automation works 
driver=webdriver.Chrome(r"C:\Users\pillap\Downloads\chromedriver_win32/chromedriver.exe")
#maximizing the window
driver.maximize_window()
#opening the olx web link through the chrome driver
driver.get('https://www.olx.in/')
time.sleep(5)
#finding the xpath for cars
car=driver.find_element_by_xpath("//div[@class='_2xg5B']/div[@class='_3AGJR _18NX_']/a[@class='_2fitb']")
#clicking on the 'cars'
car.click()
time.sleep(5)
# getting the state names from the olx website
location=driver.find_elements_by_xpath('//a[@class="_396bV _2zs4P _1nXYG"]')
prices=[]
date=[]
area=[]
company=[]
name=[]
year=[]
km=[]
state=[]
for i in range(1,len(location)):
    o=1000
    if i<=5:
        k=driver.find_element_by_xpath('//ul[@class="_3eZBP"]/li['+str(i)+']/a[@class="_396bV _2zs4P _1nXYG"]')
    else:
        j=i-5
        k=driver.find_element_by_xpath('//div[1]/li['+str(j)+']/a[@class="_396bV _2zs4P _1nXYG"]')
    #state name
    St=k.text.split()[0]
    #no of cars in that state were posted for add 
    nc=k.text.split()[1].replace('(','').replace(')','')
    #clicking on that specfic state for going to collect the car details
    k.click()
    o=o+len(prices)
    if len(prices)<10000:
        while len(prices)<o:
            # finding the car price tag
            price=driver.find_elements_by_xpath('//div[@class="_31uw8"]/span[@class="_3GOwr"]')
            for i in price:
                prices.append(i.text.split(' ')[1])
            # collecting the company name and car name tag
            cn=driver.find_elements_by_xpath('//div[@class="_31uw8"]/div[@class="_4aNdc"]')
            for i in cn:
                if i.text.split(' ')[0]=='Maruti':
                    company.append('Maruti Suzuki')
                    I=''
                    for i in i.text.split(' ')[2:]:
                        I=I+i+' '
                        I.strip()
                    name.append(I.strip())
                else:
                    company.append(i.text.split(' ')[0])
                    I=''
                    for i in i.text.split(' ')[1:]:
                        I=I+i+' '
                    name.append(I.strip())
            #finding the year and kilometers driven tag
            yk=driver.find_elements_by_xpath('//div[@class="_31uw8"]/div[@class="KFHpP"]')
            for i in yk:
                year.append(i.text.split()[0])
                km.append(i.text.split()[-2])
            #finding the area and date of add posted tags
            ad=driver.find_elements_by_xpath('//div[@class="_31uw8"]/div[@class="_1zvfB"]')
            for i in ad:
                area.append(i.text.split('\n')[0])
                state.append(St)
                try:
                    date.append(i.text.split('\n')[1])
                except IndexError:
                    date.append('no date') 
            # finding the tag for next page button
            try:
                time.sleep(5)
                np=driver.find_element_by_xpath('//button[@class="rui-39-wj rui-3evoE rui-1JPTg"]')
            except exc.NoSuchElementException:
                print("error")
                time.sleep(5)
                np=driver.find_element_by_xpath('//button[@class="rui-39-wj rui-3evoE rui-1JPTg"]')
            #clicking on the next page button
            np.click()
            time.sleep(5)
    else:
        break
        print('collected the required data')
    print(len(prices))
    #tag for reaching the top page
    try:
        driver.find_element_by_xpath('//div[@class="rui-3unfU"]').click()
    except exc.NoSuchElementException:
        time.sleep(5)
        driver.find_element_by_xpath('//div[@class="rui-3unfU"]').click()
    #tag for going to main menu of locations
    driver.find_element_by_xpath('//div[@class="_3zgcG"]').click()
# Framing the data into the dataframe
car_prices=pd.DataFrame({'company_name':company[:10000],'car_name':name[:10000],'year_of_mfg':year[:10000],'Km_driven':km[:10000],'State':state[:10000],'Area':area[:10000],'posted_price':prices[:10000]})

1054
2069
3135
4200
5234
6288
7336
8399
9463
10524


In [59]:
print(len(company),len(name),len(year),len(km),len(state),len(area),len(prices))

10804 10804 10804 10804 10844 10844 10524


In [65]:
car_prices.head()

,company_name,car_name,year_of_mfg,Km_driven,State,Area,posted_price
0,Maruti Suzuki,Swift,2009,"150,000",Maharashtra,Mharalgaon,"2,20,000"
1,Maruti Suzuki,Wagon R,2017,"19,407",Maharashtra,Powai,"4,21,000"
2,Hyundai,i20,2021,1,Maharashtra,Wadki,"7,80,563"
3,Toyota,Corolla,2006,"120,000",Maharashtra,Dharavi,"1,50,000"
4,Tata,Zest,2016,"180,000",Maharashtra,Shivaji Nagar,"3,50,000"


In [66]:
#saving the data to a file
car_prices.to_excel('used_car_prices.xlsx')

In [68]:
car_prices.describe(include='all')

,company_name,car_name,year_of_mfg,Km_driven,State,Area,posted_price
count,10000,10000,10000,10000,10000,10000,10000
unique,35,261,33,732,10,1370,583
top,Maruti Suzuki,Swift,2012,"70,000",Kerala,Ranigunj,"2,50,000"
freq,2989,522,1047,309,1106,107,158
